<a href="https://colab.research.google.com/github/EHaarer/Translating-Pedestrian-Indoor-Images-into-Maps/blob/main/GenerateLiDARVisionFigure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install torch torchvision filterpy timm lap
!pip install --upgrade scipy
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4.weights -O yolov4.weights
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4.cfg -O yolov4.cfg

import matplotlib
matplotlib.use('Agg')

!git clone https://github.com/abewley/sort.git || echo "SORT directory already exists"
!sed -i 's/matplotlib.use(.TkAgg.)/# matplotlib.use("TkAgg")/' /content/sort/sort.py

import sys
sys.path.append('/content/sort')

import cv2
import numpy as np
import torch
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from PIL import Image as PILImage
import matplotlib.pyplot as plt
from IPython.display import Image as DisplayImage, display
from scipy import stats
import math

import cv2
import numpy as np
import os


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-

In [ ]:


# Setup for YOLO
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
yolo_cfg = '/content/yolov4.cfg'
yolo_weights = '/content/yolov4.weights'
net = cv2.dnn.readNetFromDarknet(yolo_cfg, yolo_weights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)

def apply_nms(boxes, scores, confidence_threshold=0.5, nms_threshold=0.4):
    indices = cv2.dnn.NMSBoxes(boxes, scores, confidence_threshold, nms_threshold)
    if len(indices) == 0:
        return []
    indices = indices.flatten()
    return [boxes[i] for i in indices]

def process_image(image_path, depth_map_path):
    image = cv2.imread(image_path)
    depth_map = cv2.imread(depth_map_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
    Height, Width = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    detections = net.forward(net.getUnconnectedOutLayersNames())

    boxes = []
    confidences = []
    for output in detections:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if classID == 0 and confidence > 0.5:  # Detecting person
                box = detection[0:4] * np.array([Width, Height, Width, Height])
                centerX, centerY, width, height = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
    boxes = apply_nms(boxes, confidences)

    # Draw boxes and lines on the depth map
    for box in boxes:
        x, y, w, h = box
        cv2.rectangle(depth_map, (x, y), (x+w, y+h), (255), 2)  # Draw the box
        mid_x = x + w // 2
        cv2.line(depth_map, (mid_x, 0), (mid_x, Height), (255), 2)  # Draw the line

    # Save the modified depth map to Google Drive
    output_path = '/content/drive/My Drive/Adv CV files/3Personprocessed_depth_map.png'
    cv2.imwrite(output_path, depth_map)

    return image, depth_map, boxes

def main():
    input_path = '/content/drive/My Drive/Adv CV files/RefinePics/3personTest.jpg'
    depth_map_path = '/content/drive/My Drive/Adv CV files/RefinePics/3personTest.png'
    image, depth_map, boxes = process_image(input_path, depth_map_path)

if __name__ == "__main__":
    main()


##BELOW  IS THE ATTEMPT TO GENERATE THE IMAGE WITH COLOR!!! (WIP)
# import cv2
# import numpy as np
# import os

# # Setup for YOLO
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# yolo_cfg = '/content/yolov4.cfg'
# yolo_weights = '/content/yolov4.weights'
# net = cv2.dnn.readNetFromDarknet(yolo_cfg, yolo_weights)
# net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
# net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)

# def apply_nms(boxes, scores, confidence_threshold=0.5, nms_threshold=0.4):
#     indices = cv2.dnn.NMSBoxes(boxes, scores, confidence_threshold, nms_threshold)
#     if len(indices) == 0:
#         return []
#     indices = indices.flatten()
#     return [boxes[i] for i in indices]

# def process_image(image_path, depth_map_path):
#     image = cv2.imread(image_path)
#     depth_map = cv2.imread(depth_map_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
#     # Convert grayscale depth map to BGR color for drawing colored shapes
#     depth_map_color = cv2.cvtColor(depth_map, cv2.COLOR_GRAY2BGR)
#     Height, Width = image.shape[:2]
#     blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
#     net.setInput(blob)
#     detections = net.forward(net.getUnconnectedOutLayersNames())

#     boxes = []
#     confidences = []
#     for output in detections:
#         for detection in output:
#             scores = detection[5:]
#             classID = np.argmax(scores)
#             confidence = scores[classID]
#             if classID == 0 and confidence > 0.5:  # Detecting person
#                 box = detection[0:4] * np.array([Width, Height, Width, Height])
#                 centerX, centerY, width, height = box.astype("int")
#                 x = int(centerX - (width / 2))
#                 y = int(centerY - (height / 2))
#                 boxes.append([x, y, int(width), int(height)])
#                 confidences.append(float(confidence))
#     boxes = apply_nms(boxes, confidences)

#     # Draw boxes in blue and vertical lines in red on the depth map
#     for box in boxes:
#         x, y, w, h = box
#         cv2.rectangle(depth_map_color, (x, y), (x+w, y+h), (255, 0, 0), 2)  # Draw the box in blue
#         mid_x = x + w // 2
#         cv2.line(depth_map_color, (mid_x, 0), (mid_x, Height), (0, 0, 255), 2)  # Draw the line in red

#     # Save the modified depth map to Google Drive
#     output_path = '/content/drive/My Drive/processed_depth_mapCOLOR.png'
#     cv2.imwrite(output_path, depth_map_color)

#     return image, depth_map_color, boxes

# def main():
#     input_path = '/content/drive/My Drive/Adv CV files/RefinePics/3personTest.jpg'
#     depth_map_path = '/content/drive/My Drive/Adv CV files/RefinePics/3personTest.png'
#     image, depth_map_color, boxes = process_image(input_path, depth_map_path)

# if __name__ == "__main__":
#     main()

In [ ]:
import numpy as np

# Given data points
points = np.array([
    (171, 12), (168, 18), (161, 18), (214, 9), (173, 12), (212, 18),
    (179, 14), (172, 17), (161, 20), (81, 24), (166, 6), (212, 18),
    (195, 26), (192, 26)
])

# Extract x and y values
x = points[:, 0]
y = points[:, 1]

# Calculate necessary sums for the normal equations
n = len(x)
sum_x = np.sum(x)
sum_x2 = np.sum(x**2)
sum_x3 = np.sum(x**3)
sum_x4 = np.sum(x**4)
sum_y = np.sum(y)
sum_xy = np.sum(x * y)
sum_x2y = np.sum(x**2 * y)

# Set up the system of equations
A = np.array([
    [n, sum_x, sum_x2],
    [sum_x, sum_x2, sum_x3],
    [sum_x2, sum_x3, sum_x4]
])
B = np.array([sum_y, sum_xy, sum_x2y])

# Solve the system of equations
coefficients = np.linalg.solve(A, B)
a, b, c = coefficients

print(a)
print(b)
print(c)

42.517974985029554
-0.30891936050075813
0.0009017221798625724
